# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

In [29]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [23]:
def retrieve_titles(soup):
    #Your code here
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling 
    titles = [h3.find('a').attrs['title'] for h3 in book_container.findAll('h3')]
    return titles

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [24]:
def retrieve_ratings(soup):
    #Your code here
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling 
    regex = re.compile("star-rating (.*)")
    star_ratings = []
    for p in book_container.findAll('p', {"class" : regex}):
        star_ratings.append(p.attrs['class'][-1])
    star_dict = {'One': 1, 'Two': 2, 'Three':3, 'Four': 4, 'Five':5} # Manually create a dictionary to translate to numeric
    star_ratings = [star_dict[s] for s in star_ratings]
    return star_ratings

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [25]:
def retrieve_prices(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling 
    #Your code here
    prices = [p.text for p in book_container.findAll('p', class_="price_color")] 
    prices = [float(p[1:]) for p in prices] # Removing the pound sign and converting to float
    return prices

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [26]:
def retrieve_availabilities(soup):
    #Your code here
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling 
    avails = [a.text.strip() for a in book_container.findAll('p', class_="instock availability")] # Finalize the selection
    return avails

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [31]:
#Your code here
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    html_page = requests.get(url)
    
    soup = BeautifulSoup(html_page.content, 'html.parser') # Pass the page contents to beautiful soup for parsing
    titles = retrieve_titles(soup)
    prices = retrieve_prices(soup)
    ratings = retrieve_ratings(soup)
    avails = retrieve_availabilities(soup)

    print(len(titles), titles[:5])
    print(len(prices), prices[:5])
    print(len(ratings), ratings)
    print(len(avails), avails[:5])
    
df = pd.DataFrame(titles, prices, ratings, avails)
print(df.shape)


20 ['In Her Wake', 'How Music Works', 'Foolproof Preserving: A Guide to Small Batch Jams, Jellies, Pickles, Condiments, and More: A Foolproof Guide to Making Small Batch Jams, Jellies, Pickles, Condiments, and More', 'Chase Me (Paris Nights #2)', 'Black Dust']
20 [12.84, 37.32, 30.52, 25.27, 34.53]
[1, 2, 3, 5, 5, 3, 3, 3, 5, 4, 5, 3, 5, 1, 5, 3, 2, 1, 4, 2]
20 ['In stock', 'In stock', 'In stock', 'In stock', 'In stock']
20 ['Slow States of Collapse: Poems', 'Reasons to Stay Alive', 'Private Paris (Private #10)', '#HigherSelfie: Wake Up Your Life. Free Your Soul. Find Your Tribe.', 'Without Borders (Wanderlove #1)']
20 [57.31, 26.41, 47.61, 23.11, 45.07]
[3, 2, 5, 5, 2, 1, 5, 4, 4, 3, 1, 1, 3, 4, 5, 1, 1, 1, 4, 3]
20 ['In stock', 'In stock', 'In stock', 'In stock', 'In stock']
20 ['The Nameless City (The Nameless City #1)', 'The Murder That Never Was (Forensic Instincts #5)', "The Most Perfect Thing: Inside (and Outside) a Bird's Egg", 'The Mindfulness and Acceptance Workbook for Anxie

20 ["Chernobyl 01:23:40: The Incredible True Story of the World's Worst Nuclear Disaster", 'Art and Fear: Observations on the Perils (and Rewards) of Artmaking', 'A Shard of Ice (The Black Symphony Saga #1)', "A Hero's Curse (The Unseen Chronicles #1)", '23 Degrees South: A Tropical Tale of Changing Whether...']
20 [35.92, 48.63, 56.63, 50.49, 35.79]
[2, 4, 3, 3, 2, 3, 1, 2, 2, 5, 3, 4, 2, 1, 1, 1, 1, 5, 3, 5]
20 ['In stock', 'In stock', 'In stock', 'In stock', 'In stock']
20 ['Roller Girl', 'Rising Strong', 'Proofs of God: Classical Arguments from Tertullian to Barth', 'Please Kill Me: The Uncensored Oral History of Punk', 'Out of Print: City Lights Spotlight No. 14']
20 [14.1, 21.82, 54.21, 31.19, 53.64]
[5, 3, 1, 4, 5, 5, 2, 3, 2, 3, 5, 2, 5, 1, 4, 1, 1, 4, 2, 4]
20 ['In stock', 'In stock', 'In stock', 'In stock', 'In stock']
20 ["Heaven is for Real: A Little Boy's Astounding Story of His Trip to Heaven and Back", 'Future Shock (Future Shock #1)', "Ender's Game (The Ender Quintet #1

20 ['Civilization and Its Discontents', 'Cinder (The Lunar Chronicles #1)', "Catastrophic Happiness: Finding Joy in Childhood's Messy Years", 'Career of Evil (Cormoran Strike #3)', 'Breaking Dawn (Twilight #4)']
20 [59.95, 26.09, 37.35, 24.72, 35.28]
[2, 1, 2, 2, 5, 5, 3, 2, 5, 1, 1, 1, 5, 2, 2, 5, 2, 4, 3, 1]
20 ['In stock', 'In stock', 'In stock', 'In stock', 'In stock']
20 ['Annie on My Mind', 'And Then There Were None', 'A Walk in the Woods: Rediscovering America on the Appalachian Trail', 'A Visit from the Goon Squad', 'A Storm of Swords (A Song of Ice and Fire #3)']
20 [36.83, 35.01, 30.48, 14.08, 31.22]
[5, 2, 4, 5, 2, 5, 1, 1, 2, 3, 1, 3, 1, 3, 1, 4, 5, 3, 5, 1]
20 ['In stock', 'In stock', 'In stock', 'In stock', 'In stock']
20 ['Fruits Basket, Vol. 2 (Fruits Basket #2)', 'Diary of a Minecraft Zombie Book 1: A Scare of a Dare (An Unofficial Minecraft Book)', 'Y: The Last Man, Vol. 1: Unmanned (Y: The Last Man #1)', 'While You Were Mine', 'Where Lightning Strikes (Bleeding Stars

TypeError: Field elements must be 2- or 3-tuples, got ''In stock''

In [48]:


df = pd.DataFrame([titles, prices, ratings, avails]).transpose()
df.columns = ['Title', 'Price_(pounds)', 'Star_Rating', 'Availability']

df.head(30)

,Title,Price_(pounds),Star_Rating,Availability
0,Frankenstein,38,2,In stock
1,Forever Rockers (The Rocker #12),28.8,3,In stock
2,Fighting Fate (Fighting #6),39.24,3,In stock
3,Emma,32.93,2,In stock
4,"Eat, Pray, Love",51.32,3,In stock
5,Deep Under (Walker Security #1),47.09,5,In stock
6,Choosing Our Religion: The Spiritual Lives of ...,28.42,4,In stock
7,Charlie and the Chocolate Factory (Charlie Buc...,22.85,3,In stock
8,Charity's Cross (Charles Towne Belles #4),41.24,1,In stock
9,Bright Lines,39.07,5,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [ ]:
#Your code here

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!